# VGG16 com Validação Cruzada
Este notebook aplica a rede **VGG16** utilizando os dados carregados pelo `DataLoader`, avaliando o desempenho via **validação cruzada (K-Fold)**.

In [37]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import os
import cv2

from get_data_from_db import DataLoader

# Inicializa o DataLoader
data_loader = DataLoader()

# Carrega os dados de treino com labels
train_data_with_labels = data_loader.get_train_data()
print(f"Total de amostras de treino: {len(train_data_with_labels)}")

IMG_SIZE = (224, 224)  # padrão para VGG16
X, y = [], []

# Reconstruindo caminho absoluto corretamente
for filename, label in train_data_with_labels:
    if label == 0:
        filepath = os.path.join(data_loader.train_normal_path, filename)
    elif label == 1:
        filepath = os.path.join(data_loader.train_ulcerative_colitis_path, filename)
    elif label == 2:
        filepath = os.path.join(data_loader.train_polyps_path, filename)
    elif label == 3:
        filepath = os.path.join(data_loader.train_esophagitis_path, filename)
    else:
        continue

    img = cv2.imread(filepath)
    if img is None:
        print(f"[AVISO] Não foi possível carregar: {filepath}")
        continue
    
    img = cv2.resize(img, IMG_SIZE)
    X.append(img)
    y.append(label)

X = np.array(X, dtype="float32") / 255.0
y = to_categorical(np.array(y))

print(f"Formato de X: {X.shape}")
print(f"Formato de y: {y.shape}")



Dataset downloaded/cached at: C:\Users\mathe\.cache\kagglehub\datasets\francismon\curated-colon-dataset-for-deep-learning\versions\1
Total de amostras de treino: 3200
Formato de X: (3200, 224, 224, 3)
Formato de y: (3200, 4)


In [ ]:

def create_vgg_model(input_shape=(224, 224, 3), num_classes=4):
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Transfer learning (congela base)
    
    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model


In [ ]:

kf = KFold(n_splits=2, shuffle=True, random_state=42)

accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Treinando fold {fold+1}...")
    
    x_train_cv, x_val_cv = X[train_idx], X[val_idx]
    y_train_cv, y_val_cv = y[train_idx], y[val_idx]
    
    model = create_vgg_model(input_shape=(224, 224, 3), num_classes=y.shape[1])
    model.fit(x_train_cv, y_train_cv, 
              epochs=5, batch_size=32, 
              validation_data=(x_val_cv, y_val_cv), verbose=1)
    
    _, acc = model.evaluate(x_val_cv, y_val_cv, verbose=0)
    accuracies.append(acc)
    
print("Acurácias por fold:", accuracies)
print("Acurácia média:", np.mean(accuracies))


In [ ]:
# Agora teste o modelo com dados de teste
test_filenames = data_loader.get_test_data()
X_test, y_test = [], []

# Infere o label a partir do caminho, igual ao treino
for filename in test_filenames:
    if filename.startswith("test_normal"):
        label = 0
        filepath = os.path.join(data_loader.test_normal_path, filename)
    elif filename.startswith("test_ulcerative_colitis"):
        label = 1
        filepath = os.path.join(data_loader.test_ulcerative_colitis_path, filename)
    elif filename.startswith("test_polyps"):
        label = 2
        filepath = os.path.join(data_loader.test_polyps_path, filename)
    elif filename.startswith("test_esophagitis"):
        label = 3
        filepath = os.path.join(data_loader.test_esophagitis_path, filename)
    else:
        continue

    img = cv2.imread(filepath)
    if img is None:
        print(f"[AVISO] Não foi possível carregar: {filepath}")
        continue

    img = cv2.resize(img, IMG_SIZE)
    X_test.append(img)
    y_test.append(label)

X_test = np.array(X_test, dtype="float32") / 255.0
y_test = to_categorical(np.array(y_test))

print(f"Formato de X_test: {X_test.shape}")
print(f"Formato de y_test: {y_test.shape}")

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Acurácia nos dados de teste: {test_acc*100}%")


In [2]:
from get_data_from_db import DataLoader
data_loader = DataLoader()

train_data = data_loader.get_train_data()
print(f"Total de amostras de treino: {len(train_data)}")

test_data = data_loader.get_test_data()
print(f"Total de amostras de teste: {len(test_data)}")

C:\Users\mathe\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing DataLoader...
Dataset downloaded/cached at: C:\Users\mathe\.cache\kagglehub\datasets\francismon\curated-colon-dataset-for-deep-learning\versions\1
Total de amostras de treino: 3200
Total de amostras de teste: 800


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import os
import cv2

from get_data_from_db import DataLoader

# Inicializa o DataLoader
data_loader = DataLoader()

# Carrega os dados de treino com labels
train_data_with_labels = data_loader.get_train_data()
print(f"Total de amostras de treino: {len(train_data_with_labels)}")

IMG_SIZE = (224, 224)  # padrão para ResNet50
X, y = [], []

# Reconstruindo caminho absoluto corretamente
for filename, label in train_data_with_labels:
    if label == 0:
        filepath = os.path.join(data_loader.train_normal_path, filename)
    elif label == 1:
        filepath = os.path.join(data_loader.train_ulcerative_colitis_path, filename)
    elif label == 2:
        filepath = os.path.join(data_loader.train_polyps_path, filename)
    elif label == 3:
        filepath = os.path.join(data_loader.train_esophagitis_path, filename)
    else:
        continue

    img = cv2.imread(filepath)
    if img is None:
        print(f"[AVISO] Não foi possível carregar: {filepath}")
        continue
    
    img = cv2.resize(img, IMG_SIZE)
    X.append(img)
    y.append(label)

X = np.array(X, dtype="float32") / 255.0
y = to_categorical(np.array(y))

print(f"Formato de X: {X.shape}")
print(f"Formato de y: {y.shape}")

def create_resnet_model(input_shape=(224, 224, 3), num_classes=4):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Transfer learning (congela base)
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),  # melhor do que Flatten para ResNet
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

kf = KFold(n_splits=3, shuffle=True, random_state=42)

accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Treinando fold {fold+1}...")
    
    x_train_cv, x_val_cv = X[train_idx], X[val_idx]
    y_train_cv, y_val_cv = y[train_idx], y[val_idx]
    
    model = create_resnet_model(input_shape=(224, 224, 3), num_classes=y.shape[1])
    model.fit(x_train_cv, y_train_cv, 
              epochs=5, batch_size=32, 
              validation_data=(x_val_cv, y_val_cv), verbose=1)
    
    _, acc = model.evaluate(x_val_cv, y_val_cv, verbose=0)
    accuracies.append(acc)
    
print("Acurácias por fold:", accuracies)
print("Acurácia média:", np.mean(accuracies))
